# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.lesoir.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"r-article--link"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

82

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.lesoir.be/470628/article/2022-10-11/le-gouvernement-survit-lepreuve-budgetaire-de-croo-se-relance',
  'Décodage\nLe gouvernement survit à l’épreuve budgétaire, De Croo se relance'],
 ['https://www.lesoir.be/470604/article/2022-10-11/des-dizaines-de-civils-dont-un-enfant-exhumes-dans-les-villes-reconquises-par',
  'Des dizaines de civils, dont un enfant, exhumés dans les villes reconquises par l’armée ukrainienne'],
 ['https://www.lesoir.be/470526/article/2022-10-11/sauve-par-des-soldats-russes-et-des-medecins-ukrainiens-un-journaliste-italien',
  'Sauvé par des soldats russes et des médecins ukrainiens, un journaliste italien témoigne'],
 ['https://www.lesoir.be/470642/article/2022-10-11/un-vaisseau-de-la-nasa-reussi-devier-un-asteroide-de-sa-trajectoire-un-moment',
  'Un vaisseau de la Nasa a réussi à dévier un astéroïde de sa trajectoire: «Un moment décisif pour l’humanité»'],
 ['https://www.lesoir.be/470641/article/2022-10-11/ligue-des-champions-hazard-titulaire-avec-

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.lesoir.be/470628/article/2022-10-1...,Décodage\nLe gouvernement survit à l’épreuve b...
1,https://www.lesoir.be/470604/article/2022-10-1...,"Des dizaines de civils, dont un enfant, exhumé..."
2,https://www.lesoir.be/470526/article/2022-10-1...,Sauvé par des soldats russes et des médecins u...
3,https://www.lesoir.be/470642/article/2022-10-1...,Un vaisseau de la Nasa a réussi à dévier un as...
4,https://www.lesoir.be/470641/article/2022-10-1...,Ligue des champions: Hazard titulaire avec le ...
...,...,...
77,https://www.lesoir.be/468107/article/2022-09-2...,Habitat durable: vers une «non-intervention ré...
78,https://www.lesoir.be/468412/article/2022-09-2...,Un pacte à la consommation pour dépasser la cr...
79,https://www.lesoir.be/468413/article/2022-09-2...,"Au FIFF, à la découverte des jeunes talents de..."
80,https://www.lesoir.be/468491/article/2022-09-3...,Les distributeurs automatiques de pizzas s’imp...


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/lesoir_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')
chapo = soup.find("r-article--chapo").get_text(separator=' ')
content = soup.find("r-article--section").get_text(separator=' ')

print(title)
print("==================================")
print(chapo)
print("==================================")
print(content)

Le gouvernement survit à l’épreuve budgétaire, De Croo se relance
Le gouvernement Vivaldi conclut un accord budgétaire 2023-2024. Alexander De Croo décrit un exécutif qui « protège ». Il s’accroche et se relance après les rumeurs, cet été, de crise politique. Les partis de la Vivaldi et leurs présidents repartent, eux, sans trophée.
  L e Seize communiquait intensément mardi matin, avant le discours du Premier ministre à la Chambre : « Le maître mot de cet exercice budgétaire, c’est “protéger”
                     ». Alexander De Croo s’exécutera devant les élus de la nation quelques heures plus tard. Il conjuguera de toutes les façons : on protège les familles en prolongeant le paquet de base énergie ; on protège les entreprises en baissant les charges sociales ; on protège notre pays en Europe par temps de guerre en réinvestissant dans l’armée ; on protège notre Etat contre les narcotrafiquants ; on en oublie… Chacun jugera, mais le concept est là. 



### Nettoyage du texte à l'aide d'expressions régulières

In [ ]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("../data/lesoir_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{chapo}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp